In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras import models
from keras import layers
from keras import applications
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import numpy as np
import cPickle as pickle
K.set_image_dim_ordering('th')
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')
np.random.seed(6)
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
y_train = y_train / 10.
y_test = y_test / 10.
num_classes = 1
X_train = np.squeeze(X_train, axis=1)
X_test = np.squeeze(X_test, axis=1)
X_train = np.pad(X_train, ((0, 0), (10, 10), (10, 10)), 'constant')
X_test = np.pad(X_test, ((0, 0), (10, 10), (10, 10)), 'constant')

In [3]:
X_train_3D = np.stack((X_train,)*3, -1)
X_test_3D = np.stack((X_test,)*3, -1)
print(X_train_3D.shape)
X_train_3D = np.transpose(X_train_3D, (0, 3, 1, 2))
X_test_3D = np.transpose(X_test_3D, (0, 3, 1, 2))
print(X_train_3D.shape)

feature_generator = applications.VGG19(include_top=False, weights=None, input_shape=(3,48,48))
X_train_3D_features = feature_generator.predict(X_train_3D, verbose=True)
X_test_3D_features = feature_generator.predict(X_test_3D, verbose=True)
feature_shape = X_train_3D_features.shape[1] * X_train_3D_features.shape[2] * X_train_3D_features.shape[3]
X_train = X_train_3D_features.reshape(len(X_train_3D_features), feature_shape)
X_test = X_test_3D_features.reshape(len(X_test_3D_features), feature_shape)

(60000, 48, 48, 3)
(60000, 3, 48, 48)
10000/10000 [==============================] - 10s 970us/step


In [4]:
MLP = models.Sequential()
MLP.add(layers.Flatten(input_shape=feature_generator.output_shape[1:]))
MLP.add(layers.Dense(256, activation='relu', input_dim=(3,48,48)))
MLP.add(layers.Dropout(0.2))
MLP.add(layers.Dense(1, activation='linear')) # REGRESSION <--- HERE WAS 128 but for regression it should be 1!!!

In [5]:
model = keras.Model(inputs=feature_generator.input, outputs=MLP(feature_generator.output))

In [6]:
from keras import optimizers
sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['mse', 'mae']) 

In [7]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto'), \
            keras.callbacks.ModelCheckpoint('/tmp/VGG_MNIST_modelfile.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')]

In [ ]:
model.fit(X_train_3D, y_train, validation_data=(X_test_3D, y_test), callbacks=callbacks, epochs=1000, batch_size=32, verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1000
60000/60000 [==============================] - 209s 3ms/step - loss: 0.0932 - mean_squared_error: 0.0932 - mean_absolute_error: 0.2617 - val_loss: 0.0837 - val_mean_squared_error: 0.0837 - val_mean_absolute_error: 0.2533

Epoch 00001: val_loss improved from inf to 0.08366, saving model to /tmp/VGG_MNIST_modelfile.h5
Epoch 2/1000
60000/60000 [==============================] - 212s 4ms/step - loss: 0.0836 - mean_squared_error: 0.0836 - mean_absolute_error: 0.2525 - val_loss: 0.0836 - val_mean_squared_error: 0.0836 - val_mean_absolute_error: 0.2533

Epoch 00002: val_loss improved from 0.08366 to 0.08357, saving model to /tmp/VGG_MNIST_modelfile.h5
Epoch 3/1000
60000/60000 [==============================] - 215s 4ms/step - loss: 0.0836 - mean_squared_error: 0.0836 - mean_absolute_error: 0.2524 - val_loss: 0.0835 - val_mean_squared_error: 0.0835 - val_mean_absolute_error: 0.2530

Epoch 00003: val_loss improved from 0.08357 to 0.

In [ ]:
y_pred = model.predict(X_test_3D)

In [ ]:
y_pred = np.ndarray.flatten(y_pred)

In [ ]:
stats = (y_test, y_pred)
with open('Stats_Files/mnvggreg.p', 'w') as f:
    pickle.dump(stats, f)